# Задание 1

Найдите для каждого пользователя 10 продуктов, которые он заказывал чаще всего - полученный результат нужно загрузить на каггл

Обязательно сохраните себе сформированный набор, он нам пригодится дальше для нейронной сети 

In [1]:
import pandas as pd

In [3]:
products = pd.read_csv("products.csv")
transactions = pd.read_csv("transactions.csv")

In [11]:
df = transactions.groupby(["user_id", "product_id"]).agg({"product_id": "count"}).rename(columns={"product_id": "count"}).reset_index()

In [21]:
df.sort_values(["user_id", "count"], ascending=False, inplace=True)

Отсортировали по юзеру и числу покупок

In [30]:
df

,user_id,product_id,count
9459009,206209,6846,10
9459011,206209,9405,9
9459058,206209,41213,7
9459015,206209,13176,6
9459033,206209,23594,4
...,...,...,...
7,1,17122,1
6,1,14084,1
13,1,38928,1
14,1,39657,1


Для каждой покупки получим её номер в топе самых частых для конкретного юзера

In [37]:
df = df.assign(ord=df.groupby(['user_id']).cumcount())

In [43]:
df

,user_id,product_id,count,ord
9459009,206209,6846,10,0
9459011,206209,9405,9,1
9459058,206209,41213,7,2
9459015,206209,13176,6,3
9459033,206209,23594,4,4
...,...,...,...,...
7,1,17122,1,13
6,1,14084,1,14
13,1,38928,1,15
14,1,39657,1,16


Получим топ-10 для каждого юзера

In [47]:
top10_list = df[df.ord < 10].groupby("user_id").agg({"product_id": list})

In [51]:
top10_list.product_id.apply(len).drop_duplicates()

user_id
1       10
271      4
287      6
374      7
384      9
986      1
1377     2
1603     8
2700     5
3200     3
Name: product_id, dtype: int64

Как видно для некоторых юзеров не набралось 10 покупок.
Возместим их из топа самых покупаемых

In [58]:
top_20_all = list(df.groupby("product_id")["count"].agg("sum").sort_values(ascending=False).index[:20])

In [61]:
top10_list["len"] = top10_list.product_id.apply(len)

In [66]:
top10_list[top10_list.len < 10].iloc[0,:]

product_id    [11885, 40618, 39858, 36534]
len                                      4
Name: 271, dtype: object

In [83]:
def fill_10(row):
    if len(row) == 10:
        return row.product_id
    pids = row.product_id
    diff = set(top_20_all) - set(pids)
    unique_top_ranged = [p for p in top_20_all if p in diff]
    res = pids + unique_top_ranged
    return res[:10]

In [88]:
top10_list["top10"] = top10_list.apply(fill_10, axis=1)

In [92]:
sample_submission = top10_list.top10.reset_index()

In [99]:
sample_submission.top10 = sample_submission.top10.apply(lambda x: " ".join(map(str, x)))

In [101]:
sample_submission.head()

,user_id,top10
0,1,196 12427 10258 25133 46149 13032 49235 13176 ...
1,2,32792 47209 24852 19156 1559 18523 33754 16589...
2,3,39190 47766 21903 17668 9387 43961 1819 16797 ...
3,7,40852 37602 17638 21137 31683 13198 4920 30391...
4,13,27086 4210 27435 33735 1689 25134 5652 43086 4...


In [106]:
sample_submission.columns = ["user_id", "product_id"]

In [107]:
sample_submission.to_csv("sample_submission.csv", index=False)